# 1. Data Audit

In [1]:
# Support modules

import glob, os, re, random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import lasio # Las file reader module
from difflib import SequenceMatcher
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

## Import data

In [2]:
note = """
1.) The working directory should contain the data for modeling as sub-directory. 

2.) All data for modeling include well logging files (.las), deviation files (.csv) and formation top (.csv) must be separated
as sub-directory of the data directory. 
For example;
- Working directory is "Drive:/Working/".
- All data for modeling directory is "Drive:/Working/Data/".
- Well logging file directory is "Drive:/Working/Data/Well logging/" as Sub-directory of the data directory.
- Deviation file directory is "Drive:/Working/Data/Deviation/" as Sub-directory of the data directory.
- Formation top file directory is "Drive:/Working/Data/Formation top/" as Sub-directory of the data directory.

3.) Well name should be set as prefix for each file. Its name will cause file ordering and file pairing for each file of that well.
For example;
- Well name is "Well-01" (Noted: No underscore ('_') be contained in well name), so this name should be set as prefix followed by underscore ('_') for each modeling input file like this "Well-01_(...Specific name for file type indication...)"
- Example; Well logging file name, Deviation file name and Formation top file name could be "Well-01_las", "Well-01_dev" and "Well-01_top" respectively.

4.: Required data and file format;
- Well logging files include all necessary curves for 1D MEM such caliper (CAL), bitsize (BS), gamma ray (GR), density (RHOB), neutron porosity (NPHI), deep resistivity (RT), shallow resistivity (MSFL), P-sonic (DTC), S-sonic (DTS) and photoelectric factor (PEF).
- Deviation files include measured depth column named with 'MD', azimuth column named with 'AZIMUTH' and inclination or angle column named with 'ANGLE'.
- Formation top files include formation name column named with 'Formations' and top depth column named with 'Top'.
"""
print('Welcome to Automated 1D Mechanical Earth Modeling (Auto 1D MEM).')
print('Please take note on this;')
print(note)

Welcome to Automated 1D Mechanical Earth Modeling (Auto 1D MEM).
Please take note on this;

1.) The working directory should contain the data for modeling as sub-directory. 

2.) All data for modeling include well logging files (.las), deviation files (.csv) and formation top (.csv) must be separated
as sub-directory of the data directory. 
For example;
- Working directory is "Drive:/Working/".
- All data for modeling directory is "Drive:/Working/Data/".
- Well logging file directory is "Drive:/Working/Data/Well logging/" as Sub-directory of the data directory.
- Deviation file directory is "Drive:/Working/Data/Deviation/" as Sub-directory of the data directory.
- Formation top file directory is "Drive:/Working/Data/Formation top/" as Sub-directory of the data directory.

3.) Well name should be set as prefix for each file. Its name will cause file ordering and file pairing for each file of that well.
For example;
- Well name is "Well-01" (Noted: No underscore ('_') be contained in wel

In [3]:
# Setup data directory

cwd_dir_list = '\n'.join(os.listdir(os.getcwd()))

confirm = 'no'

print('According to your working directory, \n%s\nwhich one is your data directory?' %cwd_dir_list)

while confirm.lower() == 'no':
    data_folder = input('Please indicate the data directory name: ').strip()
    data_path = os.path.join(os.getcwd(), data_folder)

    if data_folder == '':
        print('\n')
        print('Please type the directory name!')
        continue

    elif os.path.isdir(data_path):
        data_dir_list = ', '.join(os.listdir(data_path))
        print('\n%s\nThese sub-directories are found.' %data_dir_list)
        print('\n')

        while True:
            print('Which one is your Well logging file directory?')
            las_folder = input('Please indicate the well logging file directory name (.las) or \'cancel\' to select a new data directory: ').strip()
            las_path = os.path.join(os.getcwd(), data_folder, las_folder)

            if las_folder == '':
                print('\n')
                print('Please type the directory name!')
                continue

            elif las_folder.lower() == 'cancel':
                break

            elif os.path.isdir(las_path):
                
                while True:
                    print('Which one is your deviation file directory?')
                    dev_folder = input('Please indicate the deviation file directory name (.csv) or \'cancel\' to select a new data directory: ').strip()
                    dev_path = os.path.join(os.getcwd(), data_folder, dev_folder)

                    if dev_folder == '':
                        print('\n')
                        print('Please type the directory name!')
                        continue

                    elif dev_folder.lower() == 'cancel':
                        break

                    elif os.path.isdir(dev_path):

                        while True:
                            print('Which one is your formation top file directory?')
                            top_folder = input('Please indicate the formation top file directory name (.csv) or \'cancel\' to select a new data directory: ').strip()
                            top_path = os.path.join(os.getcwd(), data_folder, top_folder)

                            if top_folder == '':
                                print('\n')
                                print('Please type the directory name!')
                                continue

                            elif top_folder.lower() == 'cancel':
                                break

                            elif os.path.isdir(top_path):
                                print('\n')
                                print('Gotcha!')
                                print('Your well logging file directory is: %s.' %las_path)
                                print('Your deviation file directory is: %s.' %dev_path)
                                print('Your formation top file directory is: %s.' %top_path)

                                while True:
                                    confirm = input('Are these correct? [Yes/No]: ')

                                    if confirm.lower() == 'yes':
                                        break
                                    
                                    elif confirm.lower() == 'no':
                                        break

                                    else:
                                        print('\n')
                                        print('Please confirm again!')
                                break

                            else:
                                print('\n')
                                print('Please try again, your directory \'%s\' is not found!' %top_folder)
                        break

                    else:
                        print('\n')
                        print('Please try again, your directory \'%s\' is not found!' %dev_folder)
                break

            else:
                print('\n')
                print('Please try again, your directory \'%s\' is not found!' %las_folder)

    else:
        print('\n')
        print('Please try again, your directory \'%s\' is not found!' %data_folder)

According to your working directory, 
.git
.vscode
1. Data Audit.ipynb
Automated 1D MEM.ipynb
README.md
Sirikit field
which one is your data directory?

Deviations, Formation tops, Las files, Saved files
These sub-directories are found.


Which one is your Well logging file directory?
Which one is your deviation file directory?
Which one is your formation top file directory?


Gotcha!
Your well logging file directory is: d:\Github\1D MEM\sirikit field\las files.
Your deviation file directory is: d:\Github\1D MEM\sirikit field\deviations.
Your formation top file directory is: d:\Github\1D MEM\sirikit field\formation tops.


In [4]:
# Function for pairing the data and eliminating the incompleted

def pairing_files(las_files_paths, dev_files_paths, top_files_paths):
    """
    This function is going to pairing the data (las files, dev files and top files) and disable the incompleted one.
    las_files_paths = list of las files with paths
    dev_files_paths = list of deviation files with paths
    top_files_paths = list of formation top files with paths
    """

    paired_las_files_paths = []
    paired_dev_files_paths = []
    paired_top_files_paths = []

    # pairing lAS file to deviation

    for las in las_files_paths:
        for dev in dev_files_paths:
            for top in top_files_paths:

                las_well_name = os.path.basename(las).split('_', 1)[0].lower()
                dev_well_name = os.path.basename(dev).split('_', 1)[0].lower()
                top_well_name = os.path.basename(top).split('_', 1)[0].lower()

                if las_well_name == dev_well_name == top_well_name:
                    paired_las_files_paths.append(las)
                    paired_dev_files_paths.append(dev)
                    paired_top_files_paths.append(top)

    return paired_las_files_paths, paired_dev_files_paths, paired_top_files_paths

# Generate file path

las_files = glob.glob(os.path.join(las_path, '*.las'))
dev_files = glob.glob(os.path.join(dev_path, '*.csv'))
top_files = glob.glob(os.path.join(top_path, '*.csv'))

# Pairing files lAS files, dev files and top files

las_files, dev_files, top_files = pairing_files(las_files, dev_files, top_files)

In [5]:
# Import lAS files, dev files and top files

lases = [] # Storing well logging data
df_lases = [] # Storing well logging data in panda data frame
devs = [] # Storing deviation data in panda data frame
tops = []# Storing formation top data in panda data frame

for las_file, dev_file, top_file in zip(las_files, dev_files, top_files):

    # Well logging data

    las = lasio.read(las_file)
    lases.append(las)

    # Well logging data in panda data frame

    df = las.df()
    df = df.rename_axis('MD')
    df_lases.append(df)

    # Deviation data in panda data frame

    dev = pd.read_csv(dev_file)
    devs.append(dev)

    # Fomation top data in panda data frame

    top = pd.read_csv(top_file)
    tops.append(top)

# Set directory to save files

sav_folder = 'Saved files'
sav_path = os.path.join(os.getcwd(), data_folder, sav_folder)

if not os.path.isdir(sav_path):
    os.makedirs(sav_path)

# Well names

well_names = []

for las in lases:
    well_names.append(las.well['WELL'].value)

print('The number of wells is %d.' %len(well_names))
print('Well names are %s.' %', '.join(well_names))

The number of wells is 3.
Well names are NMM-B06(BF), NMM-B07(BI), NMM-B08(BJ).


In [11]:
# Function for adjust well logging and deviation depth

def remove_gap(df_las, dev, gap)
    """
    This function can remove air gap from the total depth (MD) for both well logging and deviation data.
    df_las = las input in pandas data frame contains depth column in measured depth (MD)
    dev = Deviation survey data in pandas data frame which contains:
            1. Measured depth (MD) in column name "MD"
            2. Azimuth direction (AZIMUTH) in column name "AZIMUTH"
            3. Inclination angle (ANGLE) in column name "ANGLE"
    gap = air gap value (onshore = kelly bushing - ground level, offshore = kelly bushing)
    """

    # remove gap from df_las

    df_las = df_las.reset_index()
    df_las['MD'] = df_las['MD'] - gap
    df_las = df_las.set_index('MD')

    # remove gap from dev

    dev['MD'] = dev['MD'] - gap

    return df_las, dev

# Define field parameters to adjust (remove air gap) the well logging and deviation depth by oil field type

while True:
    field_type = input('What is this oil field type [Onshore/Offshore]: ').strip()


    if field_type.lower() == 'onshore':
        for name, df_las, dev in zip(well_names, df_lases, devs):
            print('Please type basic information for well %s' %name)

            kb = float(input('Kelly Bushing depth [Kelly bushing to sea level]: ').strip())
            gl = float(input('Ground level elevetion [Ground surface to sea level]: ').strip())
            gap = kb - gl

            df_las, dev = remove_gap(df_las, dev, gap)
        break
    
    elif field_type.lower() == 'offshore':
         
         water_levels = [] # this parameter will be used in next step.
         
         for name, df_las, dev in zip(well_names, df_lases, devs):
            print('Please type basic information for well %s' %name)

            kb = float(input('Kelly Bushing depth: ').strip())
            wl = float(input('Water depth [Sea level to seafloor]: ').strip())
            water_levels.append(wl)
            
            gap = kb

            df_las, dev = remove_gap(df_las, dev, gap)
        break
    
    else:
        print('Please type only \'Onshore\' or \'Offshore\'')
        continue



for i in range(len(well_names)):
    

    while True:
        
        
        kb = float(input('Kelly Bushing depth: ').strip())

        if field_type.lower() == 'onshore':
            gl = float(input('Ground level: ').strip())
            gap = kb - gl
            df_lases[i] = df_lases[i].reset_index()
            df_lases['MD'] = df_lases['MD'] - gap




            break

        elif field_type.lower() == 'offshore':
            wl = float(input('Water level: ').strip()) # this parameter will be used for next step
            gap = kb



            break

        else:
            print('Please type only \'Onshore\' or \'Offshore\'')
            continue


In [45]:
devs[0]['MD'] = devs[0]['MD'] - 10
devs[0]

,MD,AZIMUTH,ANGLE
0,-30.000,0.00,0.00
1,19.310,154.77,1.30
2,46.700,151.79,3.76
3,73.840,154.59,5.63
4,101.345,158.64,7.63
...,...,...,...
126,3593.690,254.72,40.43
127,3622.730,252.73,39.90
128,3651.260,254.43,37.57
129,3679.890,255.13,38.02


In [31]:
df['Height']  = df['Height'] + 10

In [21]:
df

,Name,Height,Qualification
0,Jai,15.2,Msc
1,Princi,17.4,MA
2,Gaurav,15.2,Msc
3,Anuj,15.4,Msc


In [40]:
las = df_lases[0].reset_index()
las

,MD,BIT,CLDC,GRGC,DEN,NPOR,R20T,R30T,R40T,R60T,R85T,PDPE,DT35,DTSXX_COL,DTSYY_COL
0,1035.0,12.25,8.92316,80.04658,2.25466,0.61214,20063.94531,20063.94531,20063.94531,20063.94531,20063.94531,13.58376,55.72930,NaN,NaN
1,1035.1,12.25,8.92109,81.72254,2.25267,0.63226,20062.66797,20062.66797,20062.66797,20062.66797,20062.66797,13.53716,55.73009,NaN,NaN
2,1035.2,12.25,8.91910,81.40590,2.24056,0.64422,20057.18945,20057.18945,20057.18945,20057.18945,20057.18945,13.64950,55.75632,NaN,NaN
3,1035.3,12.25,8.91740,82.10922,2.22246,0.62631,20049.98828,20049.98828,20049.98828,20049.98828,20049.98828,13.62615,55.78159,NaN,NaN
4,1035.4,12.25,8.91671,84.38502,2.20168,0.59246,20042.89453,20042.89453,20042.89453,20042.89453,20042.89453,13.80907,55.85295,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27019,3736.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27020,3737.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27021,3737.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27022,3737.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
las['MD'] = las['MD'] - 500

In [43]:
las = las.set_index('MD')

In [44]:
las

,BIT,CLDC,GRGC,DEN,NPOR,R20T,R30T,R40T,R60T,R85T,PDPE,DT35,DTSXX_COL,DTSYY_COL
MD,,,,,,,,,,,,,,
535.0,12.25,8.92316,80.04658,2.25466,0.61214,20063.94531,20063.94531,20063.94531,20063.94531,20063.94531,13.58376,55.72930,NaN,NaN
535.1,12.25,8.92109,81.72254,2.25267,0.63226,20062.66797,20062.66797,20062.66797,20062.66797,20062.66797,13.53716,55.73009,NaN,NaN
535.2,12.25,8.91910,81.40590,2.24056,0.64422,20057.18945,20057.18945,20057.18945,20057.18945,20057.18945,13.64950,55.75632,NaN,NaN
535.3,12.25,8.91740,82.10922,2.22246,0.62631,20049.98828,20049.98828,20049.98828,20049.98828,20049.98828,13.62615,55.78159,NaN,NaN
535.4,12.25,8.91671,84.38502,2.20168,0.59246,20042.89453,20042.89453,20042.89453,20042.89453,20042.89453,13.80907,55.85295,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3236.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3237.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3237.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Function for checking the curve completion of well

def check_curves(las_file, alias):
    """
    This function can check the curve completion of well.
    las_file = las file read by lasio
    alias = curve alias or alterative name of the curve 
    """

    check_curves = ['CAL', 'BS', 'GR', 'RHOB', 'NPHI', 'RT', 'MSFL', 'DTC', 'DTS', 'PEF']

    curves = [curve.mnemonic for curve in las_file.curves]

    extracted = []

    for curve in curves:
        for key, values in alias.items():
            if curve.lower() in [value.lower() for value in values]:
                extracted.append(key)

    if set(extracted) == set(check_curves):
        print('All necessary curves in well %s are completed' %las_file.well['WELL'].value)

    else:
        print('All necessary curves in well %s are incompleted.' %las_file.well['WELL'].value)
        
        if len(set(extracted).difference(set(check_curves))) == 1:
            print('Curve %s is missing.' %', '.join([curve for curve in set(check_curves) - set(extracted)]))

        else:
            print('Curves %s are missing.' %', '.join([curve for curve in set(check_curves) - set(extracted)]))

# Define curve alias for well process

alias = {
'CAL' : ['CAL', 'CALI', 'CALS', 'CLDC'],
'BS' : ['BS', 'BIT'],
'GR' : ['GR', 'GRGC', 'GAM'],
'RHOB' : ['RHOB', 'DEN', 'DENS'],
'NPHI' : ['NPHI', 'NPOR'],
'RT' : [ 'RT', 'R85T', 'LLD', 'RESD'],
'MSFL' : ['MSFL', 'R20T', 'RSHAL', 'RESS'],
'DTC' : ['DTC', 'DT35', 'DT'],
'DTS' : ['DTS', 'DTSM', 'DTSRM', 'DTSXX_COL', 'DTSYY_COL'],
'PEF' : ['PEF', 'PE', 'Pe', 'PDPE']
}

# Check available curves

print('\n')
print('Available curves for each well;')

for i in range(len(lases)):
    print(well_names[i], 'curves are: \n%s' %', '.join([c.mnemonic for c in lases[i].curves]))
    check_curves(lases[i], alias)
    print('\n')

In [ ]:
# Function for ordering formations from all well data

def merge_sequences(seq1,seq2):
    sm = SequenceMatcher(a = seq1, b = seq2)
    res = []
    
    for (op, start1, end1, start2, end2) in sm.get_opcodes():
        if op == 'equal' or op == 'delete':
            
            #This range appears in both sequences, or only in the first one.
            
            res += seq1[start1:end1]
        elif op == 'insert':
            
            #This range appears in only the second sequence.
            
            res += seq2[start2:end2]
        elif op == 'replace':
            
            #There are different ranges in each sequence - add both.
            
            res += seq1[start1:end1]
            res += seq2[start2:end2]
    
    return res

# Apply function to ordering all formation

all_forms = []

for forms in tops:
    only_forms = forms.dropna().Formations
    if all_forms == []:
        for form in only_forms:
            all_forms.append(form)
    else:
        all_forms = merge_sequences(all_forms, list(only_forms))

In [ ]:
# Function for arranging the formation following the reference one
 
def forms_arr(ref_forms, app_forms):
    """
    This function will arrange the order of the formation in list following the reference.
    ref_forms = formation order will be arranged following this reference.
    app_forms = list of the formations will be applied.
    """

    for form in ref_forms:
        if form.lower() in [form.lower() for form in app_forms]:
            app_forms.pop([form.lower() for form in app_forms].index(form.lower()))
            app_forms.append(form)
    
    return app_forms

# Function for checking formation names of the input

def check_forms(ref_forms, input_forms):
    """
    This function will check the available formation following the reference and prepare the input for the next step.
    ref_forms = available formation will be checked based on this reference.
    input_names = names of the formation 
    """

    form_names = []

    for form in ref_forms:
        if form.lower() in [name.strip().lower() for name in input_forms.split(',')]:
            form_names.append(form)
    
    return form_names  

# Function for adding the formation to the selected formation list

def add_forms(selected_forms, non_selected_forms):
    """
    This function can add more formation to selected formation list.
    selected_forms = the list of selected formations that will be added more by user.
    non_selected_forms = the list of non-selected formations
    *check_forms function is required.
    """
    
    print('Which one do you want to add more? or \'cancel\'')

    while True:
        select = input('[Comma can be used for multi-input]: ').strip()
        selected_form = check_forms(non_selected_forms, select)

        if select == '':
            print('\n')
            print('Please type formation names!')
            continue

        elif select.lower() == 'cancel':
            break

        elif selected_form == []:
            print('\n')
            print('Please try again!, formation \'%s\' is not found.' %select)
            continue

        elif set([form.lower() for form in selected_form]).issubset(set([form.lower() for form in non_selected_forms])):
            for form in selected_form:
                selected_forms.append(form)
                non_selected_forms.pop([form.lower() for form in non_selected_forms].index(form.lower()))
            break             
                 
    return selected_forms, non_selected_forms

# Function for removing the formation in the selected formation list

def remove_forms(selected_forms, non_selected_forms):
    """
    This function can remove the seleted formation.
    selected_forms = the list of selected formations that will be removed by user.
    non_selected_forms = the list of non-selected formations
    *check_forms function is required.
    """

    print('Which one do you want to remove? or \'cancel\'')

    while True:
        delete = input('[Comma can be used for multi-input]: ').strip()
        del_forms = check_forms(selected_forms, delete)

        if delete == '':
            print('\n')
            print('Please type formation names!')
            continue

        elif delete.lower() == 'cancel':
            break

        elif del_forms == []:
            print('\n')
            print('Please try again!, formation \'%s\' is not found.' %delete)
            continue       

        elif set([form.lower() for form in del_forms]).issubset(set([form.lower() for form in selected_forms])):
            for form in del_forms:
                selected_forms.pop([form.lower() for form in selected_forms].index(form.lower()))
                non_selected_forms.append(form)

            if len(del_forms) == 1:
                print('Formation %s is removed' %''.join(del_forms))
            
            else:
                print('Formation %s are removed' %', '.join(del_forms))
            break
    
    return selected_forms, non_selected_forms

In [ ]:
# Show all selectable formations

print('All formations in this field are: %s.' %', '.join(all_forms))

# Define selected formations in this project to focus

selected_forms = []

non_selected = all_forms.copy() # Be used only in this step

while True:
    print('Which one is your selected formation?')
    select = input('[Comma can be used for multi-input]: ').strip()
    selected_form = check_forms(all_forms, select)

    if select == '':
        print('Please type formation names!')
        continue

    elif selected_form == []:
        print('\n')
        print('Please try again!, formation \'%s\' is not found.' %select)
        continue

    elif set([form.lower() for form in selected_form]).issubset(set([form.lower() for form in all_forms])):
        
        for form in selected_form:
            selected_forms.append(form)
            non_selected.pop([form.lower() for form in non_selected].index(form.lower()))

        while True:

            selected_forms = forms_arr(all_forms, selected_forms)
            non_selected = forms_arr(all_forms, non_selected)        

            print('Now, only formation \'%s\' will be your selected formations' %', '.join(selected_forms))
            confirm = input('Are you okay with this? [Ok/Not]: ').strip()
            
            if confirm.lower() == 'ok':
                print('Got it, sir/ma\'am!')
                break
            
            elif confirm.lower() == 'not':
                
                while True:
                    options = input('What do you want to do? add more or edit (remove)? [Add/Remove]: ').strip()

                    if options.lower() == 'add':
                        print('\n')
                        print('The other formation in this field are: %s.' %', '.join(non_selected))
                        selected_forms, non_selected = add_forms(selected_forms, non_selected)
                        break

                    elif options.lower() == 'remove':
                        print('\n')
                        selected_forms, non_selected = remove_forms(selected_forms, non_selected)

                        if selected_forms == []:
                            print('\n')
                            print('No formation is selected!, please select formation.')
                            print('The available formation in this field are: %s.' %', '.join(all_forms))
                            selected_forms, non_selected = add_forms(selected_forms, non_selected)

                        break

                    else:
                        print('\n')
                        print('Please confirm again!')
                        continue

                continue

            else:
                print('\n')
                print('Please confirm again!')
                continue
        break

## Depth conversion

In [ ]:
# Create function for TVD computation by minimum curvature method

def tvd_mini_cuv(dev):
    """
    TVD computation function using minimum curvature survey calculation method
    dev = Deviation survey data in pandas data frame which contains:
             1. Measured depth (MD) in column name "MD"
             2. Azimuth direction (AZIMUTH) in column name "AZIMUTH"
             3. Inclination angle (ANGLE) in column name "ANGLE"
    """
    # setup parameters
    
    md = dev.MD
    prev_md = md.shift(periods = 1, fill_value = 0)
    diff_md = md - prev_md
    
    ang = dev.ANGLE
    prev_ang = ang.shift(periods = 1, fill_value = 0)
    diff_ang = ang - prev_ang
    
    azi = dev.AZIMUTH
    prev_azi = azi.shift(periods = 1, fill_value = 0)
    diff_azi = azi - prev_azi
    
    # computation
    
    cos_theta = np.cos(np.radians(diff_ang)) - (np.sin(np.radians(ang)) * np.sin(np.radians(prev_ang)) * (1 - np.cos(np.radians(diff_azi))))
    theta = np.arccos(cos_theta)
    
    rf = ((2 / theta) * np.tan(theta/2)).fillna(0)
    
    dev['TVD'] = np.cumsum((diff_md / 2) * (np.cos(np.radians(ang)) + np.cos(np.radians(prev_ang))) * rf)
    
    return dev

# Calculate TVD for all well deviations in deviation files

for dev in devs:
    tvd_mini_cuv(dev)

In [ ]:
# Generate function to convert MD to TVD in data with deviation survey data 

def tvd_interpolate(las, df_las, dev):
    """
    MD to TVD interpolation using linear interpolation method and update las file
    las = las file (.las) of the well data
    df_las = las input in pandas data frame contains depth column in measured depth (MD)
    dev = deviation survey data in pandas data frame contains depth columns in both measured depth (MD) and true vertical depth (TVD)
    """
    # Merge deviation file with well data 
    
    df_las = df_las.reset_index()
    df_las = pd.concat([dev[['MD', 'AZIMUTH', 'ANGLE', 'TVD']], df_las]).sort_values(by = ['MD']).reset_index(drop = True)
    
    # Insert true vertical depth using linear interpolation
    
    for col in df_las[['AZIMUTH', 'ANGLE', 'TVD']].columns:
        df_las[col] = df_las[col].interpolate(method = 'linear', limit_area = 'inside')
        
    # Set true vertical depth as file indices
        
    df_las = df_las.dropna(subset = ['TVD']).set_index('TVD')
    df_las = df_las.drop(list(dev['TVD']))
    
    # Update las files

    las.insert_curve(0, 'TVD', df_las.index, unit = 'm', descr = 'True Vertical Depth', value = '')
    las.insert_curve(1, 'MD', df_las.index, unit = 'm', descr = 'Measured Depth', value = '')
    las.insert_curve(2, 'AZIMUTH', df_las.index, unit = 'degree', descr = 'well Deviation in Azimuth', value = '')
    las.insert_curve(3, 'ANGLE', df_las.index, unit = 'degree', descr = 'well Deviation in Angle', value = '')
    del las.curves['DEPTH']

    print('Measured depth (MD) is converted to True vertical depth (TVD) for well %s' %las.well['WELL'].value)
    
    return las, df_las

# Imprement interpolation function

tvd_lases = []

for las, df_las, dev in zip(lases, df_lases, devs):
    las, tvd_las = tvd_interpolate(las, df_las, dev)
    tvd_lases.append(tvd_las) 

In [ ]:
for col in tvd_lases[0].columns:
    if col in alias['CAL']:
        caliper = col
    elif col in alias['BS']:
        bitsize = col

print(caliper, bitsize)

In [ ]:
# Function for create Bad Hole flag

def create_bhf(las, df_las, alias, ci):
    """
    This function can compute Bad Hole Flag using confidential interval (ci) and update las file
    las = las file (.las) of the well data
    df_las = well data in data frame 
    ci = pass data in 0.00-1.00
    * Caliper and Bitsize data are required.
    """
    
    for col in df_las.columns:
        if col in alias['CAL']:
            caliper = col
        elif col in alias['BS']:
            bitsize = col
    
    diff = df_las[caliper] - df_las[bitsize]
    interval = st.norm.interval(alpha = ci, loc = round(np.mean(diff), 2), scale = round(np.std(diff), 2))

    df_las['BHF'] = (diff.dropna() > interval[0]) & (diff.dropna() < interval[1])
    df_las['BHF'] = df_las['BHF']*1
    df_las['BHF'] ^= 1

    





# 
ci = 0.75 # can be adjusted

In [ ]:
# Plot Caliper, Bitsize and Bad hole flag

def BHF_check(wellname, top_depth, bottom_depth):
    """
    Plot histogram of caliper - bitsize difference
    """
    i = well_names.index(wellname)
    
    #create figure
    
    fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (6, 10))
    fig.suptitle(wellname, fontsize = 15, y = 1.0)
    
    #General setting for all axis
    
    for j in range(len(ax)):
        ax[j].set_ylim(top_depth, bottom_depth)
        ax[j].invert_yaxis()
        ax[j].minorticks_on() #Scale axis
        ax[j].get_xaxis().set_visible(False)
        ax[j].grid(which = 'major', linestyle = '-', linewidth = '0.5', color = 'green')
        ax[j].grid(which = 'minor', linestyle = ':', linewidth = '0.5', color = 'black') 
    
    # caliper - bitsize plot
    
    ax11 = ax[0].twiny()
    ax11.set_xlim(6,15)
    ax11.plot(wells[i].Bitsize, wells[i].index, color = 'black')
    ax11.spines['top'].set_position(('outward',0))
    ax11.set_xlabel('BS[in]',color = 'black')
    ax11.tick_params(axis = 'x', colors = 'black')
    
    ax12 = ax[0].twiny()
    ax12.set_xlim(6,15)
    ax12.plot(wells[i].Caliper, wells[i].index, color = 'grey' )
    ax12.spines['top'].set_position(('outward',40))
    ax12.set_xlabel('CALI[in]',color = 'grey')
    ax12.tick_params(axis = 'x', colors = 'grey')
    
    ax12.grid(True)
    
    # Bad Hole Flag plot
    
    ax21 = ax[1].twiny()
    ax21.plot(wells[i].BHF, wells[i].index, color = 'red')
    ax21.fill_betweenx(wells[i].index, 0, wells[i].BHF, color = 'red', label = 'Bad hole')
    ax21.spines['top'].set_position(('outward',0))
    ax21.set_xlabel('BHF', color = 'red')
    ax21.tick_params(axis = 'x', colors = 'red')
    ax21.legend(loc = 'upper right')
    
    ax21.grid(True)
    
    fig.tight_layout()
    
#     plt.savefig('Bad hole flag.png', dpi = 200, format = 'png')

In [ ]:
# Create filter interval for clipping filter with confidential interval (ci)

def filter_interval(data, ci):
    """
    Generate the filter boundary (clipping filter) with confidential interval (ci)
    define: 
    data = input
    ci = pass data in 0.00-1.00
    """
    return st.norm.interval(alpha = ci, loc = round(np.mean(data), 2), scale = round(np.std(data), 2))

# Flag using caliper log

for i in range(len(wells)):
    diff = wells[i].Caliper - wells[i].Bitsize
    interval = filter_interval(diff, 0.75) # the number can be changed
    
    print('Bad hole flag interval for well', well_names[i], ': ', interval)
    
    wells[i]['BHF'] = (diff.dropna() > interval[0]) & (diff.dropna() < interval[1])
    wells[i]['BHF'] = wells[i]['BHF']*1
    wells[i]['BHF'] ^= 1

    # update las files    

    lases[i].append_curve('BHF', wells[i]['BHF'], unit = 'unitless', descr = 'Bad Hole Flag', value = '')

In [ ]:
tvd_lases[0].columns

In [ ]:
x = ['a','b','c']
y = ['x','y','z']

for i, j in zip(x, y):
    print(i, j)

In [ ]:
'CAL' in alias['CAL']

In [ ]:
alias